# 1) Faça scraping para obter os 250 filmes com as maiores avaliações do IMDB. Existe uma página do imdb que possui essa listagem.

#### O Bloco abaixo entra na página inicial e procura o link da página que contém a lista do Top 250 Filmes

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os


# Faz a requisição da página inicial
response = requests.get('https://www.imdb.com/', headers={'User-Agent': 'Mozilla/5.0'})
soup = BeautifulSoup(response.content)
response.close()


# Faz uma lista com as tags <a>
a_tag = [tag for tag in soup.body.find_all(attrs={'role': 'menuitem'})]


# Filtra a lista para as tags que possuem href
def has_href(value):
    if value.has_attr('href'):
        return value
    else: 
        return None
a_tag = list(filter(has_href, a_tag))


# Filtra a lista para deixar apenas a que contém o link para a página dos Top250 Filmes
def taking_href(value):
    if value['href'] == '/chart/top/?ref_=nv_mv_250':
        return value
    else:
        None
a_tag = list(filter(taking_href, a_tag))


# Atribui o link dos Top250 a uma variavel
href_link = a_tag[0]['href']

#### Abaixo, o algoritmo acessa a página dos Top 250 Filmes e colocas eles com seus links em um dicionário.

In [2]:
# Faz a requisição do da página dos Top250 Filmes
response = requests.get('https://www.imdb.com' + href_link, headers={'User-Agent': 'Mozilla/5.0'})
soup = BeautifulSoup(response.content)
response.close()

# Obtém as tags que cotém o nome e o link de cada filme e coloca em um dicionário
a_tag = [tag.a for tag in soup.body.find_all(attrs={'class': 'titleColumn'})]
names = [tag.text for tag in a_tag]
links = [tag['href'].split('?')[0] for tag in a_tag]
names_and_links = dict(zip(names, links))

display(names_and_links)

{'Um Sonho de Liberdade': '/title/tt0111161/',
 'O Poderoso Chefão': '/title/tt0068646/',
 'Batman: O Cavaleiro das Trevas': '/title/tt0468569/',
 'O Poderoso Chefão II': '/title/tt0071562/',
 '12 Homens e uma Sentença': '/title/tt0050083/',
 'A Lista de Schindler': '/title/tt0108052/',
 'O Senhor dos Anéis: O Retorno do Rei': '/title/tt0167260/',
 'Pulp Fiction: Tempo de Violência': '/title/tt0110912/',
 'O Senhor dos Anéis: A Sociedade do Anel': '/title/tt0120737/',
 'Três Homens em Conflito': '/title/tt0060196/',
 'Forrest Gump: O Contador de Histórias': '/title/tt0109830/',
 'Clube da Luta': '/title/tt0137523/',
 'O Senhor dos Anéis: As Duas Torres': '/title/tt0167261/',
 'A Origem': '/title/tt1375666/',
 'O Império Contra-Ataca': '/title/tt0080684/',
 'Matrix': '/title/tt0133093/',
 'Os Bons Companheiros': '/title/tt0099685/',
 'Um Estranho no Ninho': '/title/tt0073486/',
 'Seven: Os Sete Crimes Capitais': '/title/tt0114369/',
 'A Felicidade Não se Compra': '/title/tt0038650/',
 '

# 2) Faça scraping das páginas específicas dos 250 filmes obtidos no item anterior. Devem ser obtidos: Título, Ano de lançamento, url do poster, imagem do poster, nota imdb, lista de gêneros e lista de diretores. Trate os casos nos quais não existirem essas informações.

In [94]:
# Cria um dataframe para transforma-lo em json posteriormente
columns = ['titulo', 'ano_lancamento', 'url_poster', 'img_poster', 'nota_imdb', 'generos', 'diretores']
df = pd.DataFrame(columns=columns)

idx = 0

# Loop que faz a respagem de dados na página de cada um dos 250 filmes
for name, link in names_and_links.items():
    # Faz as requisições
    response = requests.get('https://www.imdb.com' + link, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.content)
    response.close()


    # Pega os dados
    title = name
    year = list(map(lambda x: x.text, soup.body.find_all(attrs={'class', 'ipc-link ipc-link--baseAlt ipc-link--inherit-color'})))[-2]
    img_link = soup.body.find_all(attrs={'class': 'ipc-media ipc-media--poster-27x40 ipc-image-media-ratio--poster-27x40 ipc-media--baseAlt ipc-media--poster-l ipc-poster__poster-image ipc-media__img'})[0].contents[0]['src']
    directors = soup.body.find(attrs={'class': 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}).text

    try:
        grade = soup.body.find_all(attrs={'class': 'sc-bde20123-1 iZlgcd'})[0].text
    except IndexError:
        grade = ''

    try:
        genres = list(map(lambda x: x.text, soup.body.find_all(attrs={'class': 'sc-2971dade-4 fCmvlv'})[0].contents[0].find_all('a')))
    except IndexError:
        genres = ''
    img_name = title.replace(' ', '_') + '.jpg'
    if ':' in title:
        title.replace(':', '')
        
    # Baixa a imagens na pasta images

    
    
    
    path = os.path.join(os.getcwd(), 'images')
    goes_to = os.path.join(path, img_name)
    img = requests.get(img_link)
    with open(goes_to, 'wb') as f:
        f.write(img.content)
    

    # Acrescenta uma nova linha ao dataframe
    #new_row = pd.DataFrame(dict(zip(columns, [name, year, img_link, 'images/' + f'{name}.jpg', grade, genres, directors])), index=[0])
    #df = pd.concat([df, new_row], ignore_index=True)

    if idx == 5:
        break

    #display(dict(zip(columns, [name, year, img_link, 'images/' + f'{name}.jpg', grade, genres, directors])))

    print([name, year, img_link, 'images/' + f'{img_name}', grade, genres, directors])


    
    

    


    

['Um Sonho de Liberdade', '1994', 'https://m.media-amazon.com/images/M/MV5BNDE3ODcxYzMtY2YzZC00NmNlLWJiNDMtZDViZWM2MzIxZDYwXkEyXkFqcGdeQXVyNjAwNDUxODI@._V1_QL75_UX190_CR0,2,190,281_.jpg', 'images/Um_Sonho_de_Liberdade.jpg', '9.3', ['Drama'], 'Frank Darabont']
['O Poderoso Chefão', '1972', 'https://m.media-amazon.com/images/M/MV5BZjgwNzE5ODgtYzAyZC00YTZhLThhNDktMDVlOGNhYzk5NTVkXkEyXkFqcGdeQXVyMTAzMDM4MjM0._V1_QL75_UY281_CR4,0,190,281_.jpg', 'images/O_Poderoso_Chefão.jpg', '9.2', ['Crime', 'Drama'], 'Francis Ford Coppola']
['Batman: O Cavaleiro das Trevas', '2008', 'https://m.media-amazon.com/images/M/MV5BM2E1ZjEyYWQtMjgyMy00ZTkwLThkM2QtN2YwM2NmYzAyYjE0XkEyXkFqcGdeQXVyMTAzMDM4MjM0._V1_QL75_UY281_CR11,0,190,281_.jpg', 'images/Batman:_O_Cavaleiro_das_Trevas.jpg', '9.0', ['Action', 'Crime', 'Drama'], 'Christopher Nolan']
['O Poderoso Chefão II', '1974', 'https://m.media-amazon.com/images/M/MV5BOGM2M2Q0ZWQtZjc5NC00NDY4LWIxYTktMDE4ZjIyYTRlNTIyXkEyXkFqcGdeQXVyODc0OTEyNDU@._V1_QL75_UY281_CR6,0,

KeyboardInterrupt: 

In [96]:
print('Matheus:Maia'.replace(':', ''))

MatheusMaia
